# Mcpy
## MolSSI Summer School -- Domain Specific Project
### -- Van Nguyen, Brian Nguyen, Joel Ricci, Sabiha Rustam, and Brandon Butler

### 2019-07-31

### Import necessary modules

In [41]:
import numpy as np
import mcpy.particles
import mcpy.box
import mcpy.integrator
import mcpy.mcsimulation
from timeit import default_timer as timer

#### Import both the pure python and c++/python pairwise modules

In [ ]:
import mcpy.pairwise
import mccpp.pairwise

### Set up simulation parameters

In [27]:
reduced_temperature = 0.9
reduced_density = 0.9
num_part = 500

## Initialize objects

* Box -- holds the box dimensions and can manipulate periodic
  points
* Particles -- holds the particle's coordinates and number
  with the potential to later aggregate other quantities
* LJ -- computes the pairwise potential of a series of
  points
* Integrator -- Class that actually performs the displacement
  trial move
* MCSimulation -- Interfaces all the classes correctly together.

In [28]:
box_dims = np.full(3, np.cbrt(num_part / reduced_density))
box = mcpy.box.Box(box_dims=box_dims)
part = mcpy.particles.Particles.from_random(num_particles=num_part,
                                            box_dims=box.box_dims)
# Python Object
lj_py = mcpy.pairwise.LJ(cutoff=3.)
# C++/PYBIND Object
lj_cpp = mccpp.pairwise.LJ(cutoff=3.)

intg = mcpy.integrator.Integrator(1/reduced_temperature)

## Python Implementation
### Create and add objects to MCSimulation object

In [31]:
mc_1 = mcpy.mcsimulation.MCSimulation()

mc_1.add_integrator(intg)
mc_1.add_box(box)
mc_1.add_particles(part)

### Adding the python LJ object

In [33]:
mc_1.add_potential(lj_py)

### Py LJ class: Runs Simulation for 100,000 steps

In [34]:
t1 = timer()
mc_1.run(100000)
t2 = timer()

Step 10000, Energy 14.930766960348535, Acceptance Rates [0.4154]
Step 20000, Energy -3.1001964247203952, Acceptance Rates [0.37065]
Step 30000, Energy -5.12895092622224, Acceptance Rates [0.3679]
Step 40000, Energy -5.573390514143759, Acceptance Rates [0.3846]
Step 50000, Energy -5.730004343753291, Acceptance Rates [0.396]
Step 60000, Energy -5.811393549070274, Acceptance Rates [0.40588333]
Step 70000, Energy -5.9200255561047435, Acceptance Rates [0.41298571]
Step 80000, Energy -5.968260560632954, Acceptance Rates [0.41845]
Step 90000, Energy -6.002809843398372, Acceptance Rates [0.42507778]
Step 100000, Energy -5.99055150947623, Acceptance Rates [0.42697]


In [35]:
print("Python LJ Class: 100,000 steps in {:4d} sec".format(int(t2 - t1)))
print("{:5d} steps per second.".format(int(1e6/(t2 - t1))))

Python LJ Class: 100,000 steps in   22 sec
43899 steps per second.


## C++ Implementation
### Create and add objects to MCSimulation object

In [36]:
mc_2 = mcpy.mcsimulation.MCSimulation()

mc_2.add_integrator(intg)
mc_2.add_box(box)
mc_2.add_particles(part)

### Adding the C++ LJ object

In [37]:
mc_2.add_potential(lj_cpp)

### C++ LJ class: Runs Simulation for 100,000 steps

In [38]:
t1 = timer()
mc_2.run(100000)
t2 = timer()

Step 10000, Energy -6.113956233473307, Acceptance Rates [0.4018]
Step 20000, Energy -6.04461480800406, Acceptance Rates [0.4022]
Step 30000, Energy -6.1033576545226325, Acceptance Rates [0.40386667]
Step 40000, Energy -6.107326511027829, Acceptance Rates [0.403575]
Step 50000, Energy -6.148701738306221, Acceptance Rates [0.40304]
Step 60000, Energy -6.136297435702068, Acceptance Rates [0.40366667]
Step 70000, Energy -6.134261982488871, Acceptance Rates [0.40352857]
Step 80000, Energy -6.128953833951505, Acceptance Rates [0.4047125]
Step 90000, Energy -6.166120698390126, Acceptance Rates [0.40544444]
Step 100000, Energy -6.157465487534668, Acceptance Rates [0.40571]


In [39]:
print("C++ LJ Class: 100,000 steps in {:4d} sec".format(int(t2 - t1)))
print("{:5d} steps per second.".format(int(1e6/(t2 - t1))))

C++ LJ Class: 100,000 steps in   43 sec
23110 steps per second.


### Comparing

In [40]:
print("Numpy code is {} times faster.".format(43899/23110))

Numpy code is 1.8995672868887927 times faster.
